In [ ]:
!pip install evidently

In [48]:
import evidently
print(evidently.__version__)

0.7.7


In [1]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-03.parquet

--2025-06-17 08:37:18--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-03.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 108.138.245.225, 108.138.245.58, 108.138.245.96, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|108.138.245.225|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1372372 (1.3M) [binary/octet-stream]
Saving to: ‘green_tripdata_2024-03.parquet’

green_tripdata_2024 100%[===================>]   1.31M  1.53MB/s    in 0.9s    

2025-06-17 08:37:20 (1.53 MB/s) - ‘green_tripdata_2024-03.parquet’ saved [1372372/1372372]



In [59]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from evidently import Report
from evidently.metrics.regression import RMSE, MAE, R2Score

In [13]:
def preprocess_trip_data(df):

  df['lpep_dropoff_datetime'] = pd.to_datetime(df['lpep_dropoff_datetime'])
  df['lpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])

  categorical = ['PULocationID', 'DOLocationID']
  df[categorical] = df[categorical].astype(str)

  return df

In [11]:
def add_trip_duration(df):

  df["Duration"] = df['lpep_dropoff_datetime'] - df['lpep_dropoff_datetime']
  df["Duration"] = df["Duration"].apply(lambda td: td.total_seconds() / 60)

  return df

In [35]:
df = pd.read_parquet('/content/green_tripdata_2024-03.parquet')

In [36]:
df = preprocess_trip_data(df)

In [37]:
df.columns

Index(['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime',
       'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID',
       'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge',
       'total_amount', 'payment_type', 'trip_type', 'congestion_surcharge'],
      dtype='object')

In [38]:
df.shape

(57457, 20)

In [39]:
df = add_trip_duration(df)

In [40]:
dv = DictVectorizer()
categorical = ['PULocationID', 'DOLocationID']

train_dict = df[categorical].to_dict(orient="records")
X_train = dv.fit_transform(train_dict)
y_train = df['Duration'].values
print(len(dv.feature_names_))

463


In [41]:
reg = LinearRegression().fit(X_train, y_train)

In [56]:
f_result = pd.DataFrame({
    "target": y_train,
    "prediction": reg.predict(X_train)
})

report = Report(metrics=[
    RMSE(),
    MAE(),
    R2Score(),
])

report.run(reference_data=df, current_data=df)
report.show(mode="inline")

/usr/local/lib/python3.11/dist-packages/evidently/core/datasets.py:866: RuntimeWarning: invalid value encountered in scalar divide
  infinite=StatCountValue(infinite_count, infinite_count / data.count()),
/usr/local/lib/python3.11/dist-packages/evidently/core/datasets.py:866: RuntimeWarning: invalid value encountered in scalar divide
  infinite=StatCountValue(infinite_count, infinite_count / data.count()),


ValueError: No default regression in data definition